In [32]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# read and look at data
edu = pd.read_csv('https://raw.githubusercontent.com/wchen952/STAT-303-2-Project/main/data/xAPI-Edu-Data.csv')
edu_test = pd.read_csv("https://raw.githubusercontent.com/wchen952/STAT-303-2-Project/main/data/edu_test.csv")

In [35]:
# make train dataset
edu_train = edu.copy()

In [36]:
# clean data 

# get two grades
edu_train = edu_train.loc[edu['GradeID'].isin(['G-07','G-08'])]

# drop stageID, sectionID, semester, relation, parentansweringsurvey, parentschoolsatisfaction
edu_train.drop(['NationalITy', 'PlaceofBirth', 'StageID', 'SectionID', 'Semester', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction'], inplace = True, axis = 1)

#dropping na values
edu_train.dropna(inplace =True)

# drop English topic because there are only two observations 
edu_train = edu_train[edu_train.Topic != 'English']

# drop low
edu_train = edu_train[edu_train.Class != 'L']

#replacing M with 0 and H with 1
edu_train["Class"].replace(to_replace = {'M': 0, 'H': 1}, inplace = True)



In [37]:
edu_train['Topic'].value_counts()

Biology      26
Geology      24
Arabic       22
Spanish      17
Chemistry    16
IT           13
Science      13
Quran        13
History      10
Math          9
Name: Topic, dtype: int64

In [38]:
edu_train['Class'].value_counts()

0    101
1     62
Name: Class, dtype: int64

In [39]:
# developing model
model1 = smf.logit(formula = 'Class~gender+raisedhands+VisITedResources+AnnouncementsView+Discussion+StudentAbsenceDays', data = edu_train).fit()
model1.summary()

Optimization terminated successfully.
         Current function value: 0.442542
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Class   No. Observations:                  163
Model:                          Logit   Df Residuals:                      156
Method:                           MLE   Df Model:                            6
Date:                Tue, 28 Feb 2023   Pseudo R-squ.:                  0.3338
Time:                        13:32:55   Log-Likelihood:                -72.134
converged:                       True   LL-Null:                       -108.27
Covariance Type:            nonrobust   LLR p-value:                 1.395e-13
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -8.8299      1.708     -5.171      0.000     -12.177      -5.483
gender[T.M]                      -1.3045      0.476     -2.740      0.006      -2.238      -0.371
StudentAbsenceDays[T.Under-7]     2.2674      0.796      2.849      0.004       0.707       3.827
raisedhands                       0.0250      0.012      2.158      0.031       0.002       0.048
VisITedResources                  0.0550      0.017      3.148      0.002       0.021       0.089
AnnouncementsView                 0.0014      0.010      0.134      0.893      -0.019       0.021
Discussion                        0.0204      0.008      2.439      0.015       0.004       0.037
=================================================================================================
"""

In [40]:
# make test dataset
edu_test = edu.copy()

In [41]:
# predict
pred_class = model1.predict(edu_test)
pred_class

0      0.002024
1      0.003166
2      0.000138
3      0.000684
4      0.004743
         ...   
475    0.000245
476    0.380833
477    0.379802
478    0.002570
479    0.002769
Length: 480, dtype: float64

In [42]:
edu

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,F,Jordan,Jordan,MiddleSchool,G-08,A,Chemistry,S,Father,5,4,5,8,No,Bad,Above-7,L
476,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,F,Father,50,77,14,28,No,Bad,Under-7,M
477,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,S,Father,55,74,25,29,No,Bad,Under-7,M
478,F,Jordan,Jordan,MiddleSchool,G-08,A,History,F,Father,30,17,14,57,No,Bad,Above-7,L
